## VFDB 下游分析

#### iHMP-HSM7J4QT-setB

In [4]:
%%bash 
ls ~/Lab/Custom-DataBase/VFDB/Test-DataBase/iHMP-HSM7J4QT-setB

HSM7J4QT_genefamilies.tsv
HSM7J4QT_humann2_temp
HSM7J4QT_pathabundance.tsv
HSM7J4QT_pathcoverage.tsv


In [5]:
!head ~/Lab/Custom-DataBase/VFDB/Test-DataBase/iHMP-HSM7J4QT-setB/HSM7J4QT_genefamilies.tsv

# Gene Family	HSM7J4QT_Abundance-RPKs
UNMAPPED	7879093.0000000000
tuf	311.2549367516
tuf|g__Mycoplasma.s__mycoides	97.2894360701
tuf|g__Mycoplasma.s__penetrans	52.9389852550
tuf|g__Mycoplasma.s__hyopneumoniae	35.7590193883
tuf|g__Mycoplasma.s__synoviae	32.6786073565
tuf|g__Mycoplasma.s__mobile	26.3589318648
tuf|g__Mycoplasma.s__pulmonis	19.7239209818
tuf|g__Mycoplasma.s__gallisepticum	19.3621437388


### id mapping

如何将VF family对应/归类为VF 相关功能

### Genus level gene families and pathways
By default, the gene families and pathways output files from HUMAnN2 are species level. To obtain genus level gene families and pathways, follow these steps.

* Create a genus level gene families file

`$ humann2_gene_families_genus_level --input $SAMPLE_genefamilies.tsv --output $SAMPLE_genefamilies_genus_level.tsv`
In this command, replace `$SAMPLE_genefamilies.tsv` with the species level gene families file created by default by HUMAnN2 and `$SAMPLE_genefamilies_genus_level.tsv` with the name of the gene families genus level file that will be created.

* Run HUMAnN2, with the genus level gene families file as input, to get genus level pathways output files

`$ humann2 --input $SAMPLE_genefamilies_genus_level.tsv --output humann2_genus_level_output`
This run will be much faster and require less memory than the original run as HUMAnN2 is provided gene family abundances so it only needs to compute the pathways.


输出的为genus level 且与pathways无关，可尝试用：   
```shell
humann2_gene_families_genus_level --input $SAMPLE_genefamilies.tsv --output $SAMPLE_genefamilies_genus_level.tsv
```

## Manipulating HUMAnN2 output tables

### Attaching names to features

```shell
humann2_rename_table --input demo_fastq/demo_genefamilies.tsv --output demo_fastq/demo_genefamilies-names.tsv --names uniref90
```

不知能否对VFDB加上注释，`--names`参数怎么调？

### Normalizing RPKs to relative abundance

```shell
humann2_renorm_table --input demo_fastq/demo_genefamilies.tsv --output demo_fastq/demo_genefamilies-cpm.tsv --units cpm --update-snames
```

## 结合 MetaPhlAn2 物种组成分析

### Visualizing stratified HUMAnN2 output

```shell
humann2_barplot --input hmp_pathabund.pcl --focal-feature METSYN-PWY --focal-metadatum STSite --last-metadatum STSite --output plot1.png
```

[MetaPhlAn2 tutorial](https://bitbucket.org/biobakery/biobakery/wiki/metaphlan2#rst-header-id31)

* Create a heatmap with hclust2   
* Create a cladogram with GraPhlAn   
* Create a strain-level marker-based heatmap (PanPhlAn)

* [LEfSe Tutorial](https://bitbucket.org/biobakery/biobakery/wiki/lefse#rst-header-visualization)

* [MaAsLin Tutorial](https://bitbucket.org/biobakery/biobakery/wiki/maaslin)

按物种分VF 

## Gene family 分析

#### humann2_join_tables

In [ ]:
humann2_join_tables -i 763577454 -o 763577454_genefamilies.tsv --file_name genefamilies

In [ ]:
humann2_join_tables -i health -o health_genefamilies.tsv --file_name genefamilies

#### humann2_renorm_table

In [ ]:
humann2_renorm_table -i 763577454_genefamilies.tsv -o 763577454_genefamilies_cpm.tsv --units cpm

#### Associating HUMAnN2 functions with metadata

* LEfSe (for simple study designs)   
* MaAsLin (for complex study designs)   
* HAllA (for high-dimensional metadata)   

When associating metadata with HUMAnN2 features, it is often beneficial to associate with community totals and avoid testing each individual feature stratification (to improve statistical power). This is the approach used by the HUMAnN2 utility script humann2_associate, which can compare feature totals across samples with 
1) a single continuous metadatum (via the Spearman Correlation) or     
2) a single categorical metadatum (via the Kruskal-Wallis H-test).

```R
# First read in the metadata file:
in_meta <- read.table("/home/ubuntu/CourseData/metagenomics/Module4/module4_metadata.txt", header=T, sep=" ", stringsAsFactors = FALSE)

# Set the rownames to be the sample labels
rownames(in_meta) <- in_meta$sample

# Transpose this table:
in_meta_t <- data.frame(t(in_meta), stringsAsFactors = FALSE)

# Now read in the pathway abundances (in copies per million):
in_path <- read.table("/path/to/humann2_final_out/humann2_pathabundance_relab.tsv", header=T, sep='\t', stringsAsFactors = FALSE, quote="", comment.char="", row.names=1)

# Remove "_Abundance" from the column names:
colnames(in_path) <- gsub("_Abundance", "", colnames(in_path))

# Order the sample columns to be the same as in the metadata table:
in_path <- in_path[,colnames(in_meta_t)]

# Combine these two tables by matching columns:
out_pcl <- rbind(in_meta_t, in_path)

# Write out this combined table of metadata and pathway abundances (note you'll need to change the path):
write.table(x = out_pcl, file = "/path/to/out/humann2_pathabundance_relab_meta.pcl", col.names=FALSE, row.names=TRUE, sep="\t", quote=FALSE)
```

https://github.com/LangilleLab/microbiome_helper/wiki/CBW-2018-Metagenomic-Taxonomic-and-Functional-Composition-Tutorial#filtering-out-low-quality-reads

#### humann2_barplot

In [ ]:
humann2_barplot --sort sum metadata --input humann2_pathabundance_relab_meta.pcl --focal-feature ARGININE-SYN4-PWY --focal-metadatum disease_state --last-metadatum disease_state --output ARGININE-SYN4-PWY_by_disease.png

humann2_barplot 参数设置

#### Visualizing stratified HUMAnN2 output

https://bitbucket.org/biobakery/biobakery/wiki/humann2#rst-header-id28

相对丰度前15/20的基因相对丰度柱形图

## Visualization in QIIME2

### Convert the table to biom

```shell
biom convert -i /projects/e30740/<yourfolder>/humann2/allgenefamilies_ko_cpm_stratified.tsv -o /projects/e30740/<yourfolder>/humann2/allgenefamilies_ko_cpm_stratified.biom --table-type "Gene table" --to-hdf5

biom convert -i /projects/e30740/<yourfolder>/humann2/allpathabundance_relab_stratified.tsv -o /projects/e30740/<yourfolder>/humann2/allpathabundance_relab_stratified.biom --table-type "Pathway table" --to-hdf5
```

In [ ]:
humann2_split_stratified_table --input 

In [8]:
import pandas as pd
vfid = pd.read_csv("~/Lab/Custom-DataBase/VFDB/g-vfid.csv", header = 0)

In [15]:
vfid

,plc,VF0470
0,plc,VF0470
1,plcD,VF0469
2,basJ,VF0467
3,basI,VF0467
4,basH,VF0467
...,...,...
32516,YPK_3865,CVF869
32517,YPTB3285,CVF870
32518,YpsIP31758_0695,CVF870
32519,YPTS_3422,CVF870


In [26]:
g2vf = dict(zip(vfid.plc, vfid.VF0470))

In [41]:
g2vf['plcD']

'CVF304'

In [37]:
g2vf

{'plc': 'TX160',
 'plcD': 'CVF304',
 'basJ': 'CVF641',
 'basI': 'CVF768',
 'basH': 'CVF768',
 'barB': 'CVF768',
 'barA': 'CVF768',
 'basG': 'CVF768',
 'basF': 'CVF768',
 'entE': 'CVF477',
 'basD': 'CVF768',
 'basC': 'CVF768',
 'bauA': 'IA004',
 'bauB': 'IA004',
 'bauE': 'IA004',
 'bauC': 'IA004',
 'bauD': 'IA004',
 'basB': 'CVF768',
 'basA': 'CVF768',
 'bauF': 'CVF768',
 'csuA/B': 'CVF770',
 'csuA': 'CVF770',
 'csuB': 'AI028',
 'csuC': 'AI028',
 'csuD': 'AI028',
 'csuE': 'CVF770',
 'bap': 'CVF771',
 'pgaA': 'CVF772',
 'pgaB': 'CVF772',
 'pgaC': 'CVF772',
 'pgaD': 'CVF772',
 'adeF': 'CVF773',
 'adeG': 'CVF773',
 'adeH': 'CVF773',
 'ompA': 'CVF806',
 'abaI': 'CVF777',
 'abaR': 'CVF777',
 'bfmR': 'CVF778',
 'bfmS': 'CVF778',
 'exeA': 'CVF780',
 'exeB': 'CVF780',
 'exeC': 'CVF780',
 'exeD': 'CVF780',
 'exeE': 'CVF780',
 'exeF': 'CVF780',
 'exeG': 'CVF780',
 'exeH': 'CVF780',
 'exeI': 'CVF780',
 'exeJ': 'CVF780',
 'exeK': 'CVF780',
 'exeL': 'CVF780',
 'exeM': 'CVF780',
 'exeN': 'CVF780',
 '

In [10]:
gf = pd.read_csv("~/Lab/Custom-DataBase/VFDB/humann2_genefamilies_stratified.tsv", sep="\t" )

In [23]:
gf

,Acc,Feature,Taxa,ERR695598_Abundance-RPKs,ERR695599_Abundance-RPKs,ERR695600_Abundance-RPKs,ERR695601_Abundance-RPKs,ERR695602_Abundance-RPKs,ERR695603_Abundance-RPKs,ERR695604_Abundance-RPKs,...,ERR695628_Abundance-RPKs,ERR695629_Abundance-RPKs,ERR695630_Abundance-RPKs,ERR695631_Abundance-RPKs,ERR695632_Abundance-RPKs,ERR695633_Abundance-RPKs,ERR695634_Abundance-RPKs,ERR695635_Abundance-RPKs,ERR695636_Abundance-RPKs,ERR695637_Abundance-RPKs
0,UNGROUPED,A225_1326,g__Klebsiella.s__oxytoca,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.323311,0.000000,0.333667,0.000000
1,UNGROUPED,A225_1327,g__Klebsiella.s__oxytoca,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2,UNGROUPED,A225_1596,g__Klebsiella.s__oxytoca,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
3,UNGROUPED,A225_1597,g__Klebsiella.s__oxytoca,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
4,UNGROUPED,A225_1598,g__Klebsiella.s__oxytoca,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.932836,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2089,UNGROUPED,yhxB/manB,g__Haemophilus.s__somnus,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,1.514147,0.840011,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2090,UNGROUPED,yijP,g__Escherichia.s__coli,0.0,0.0,0.000000,0.506664,0.0,0.0,0.0,...,0.000000,0.000000,0.279188,0.0,0.0,0.0,0.613874,0.000000,0.000000,0.000000
2091,UNGROUPED,ykgK/ecpR,g__Escherichia.s__coli,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2092,UNGROUPED,zmp1,g__Mycobacterium.s__avium,0.0,0.0,0.543774,0.000000,0.0,0.0,0.0,...,3.191224,2.666430,0.533049,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.528821


In [21]:
pd.merge(gf, vfid, left_on='Feature', right_on='plc', how='left')

,Acc,Feature,Taxa,ERR695598_Abundance-RPKs,ERR695599_Abundance-RPKs,ERR695600_Abundance-RPKs,ERR695601_Abundance-RPKs,ERR695602_Abundance-RPKs,ERR695603_Abundance-RPKs,ERR695604_Abundance-RPKs,...,ERR695630_Abundance-RPKs,ERR695631_Abundance-RPKs,ERR695632_Abundance-RPKs,ERR695633_Abundance-RPKs,ERR695634_Abundance-RPKs,ERR695635_Abundance-RPKs,ERR695636_Abundance-RPKs,ERR695637_Abundance-RPKs,plc,VF0470
0,UNGROUPED,A225_1326,g__Klebsiella.s__oxytoca,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.323311,0.000000,0.333667,0.000000,A225_1326,CVF859
1,UNGROUPED,A225_1327,g__Klebsiella.s__oxytoca,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,A225_1327,CVF859
2,UNGROUPED,A225_1596,g__Klebsiella.s__oxytoca,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,A225_1596,CVF849
3,UNGROUPED,A225_1597,g__Klebsiella.s__oxytoca,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,A225_1597,CVF849
4,UNGROUPED,A225_1598,g__Klebsiella.s__oxytoca,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.932836,0.000000,0.000000,A225_1598,CVF849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22608,UNGROUPED,zmp1,g__Mycobacterium.s__avium,0.0,0.0,0.543774,0.0,0.0,0.0,0.0,...,0.533049,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.528821,zmp1,CVF655
22609,UNGROUPED,zmpB,g__Streptococcus.s__pneumoniae,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,zmpB,CVF146
22610,UNGROUPED,zmpB,g__Streptococcus.s__pneumoniae,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,zmpB,CVF146
22611,UNGROUPED,zmpB,g__Streptococcus.s__pneumoniae,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,zmpB,CVF146
